<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/main/MVA2024_ex12notebookC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2024 ex12notebookC

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2024

----
## 演習課題: 手書き数字識別
---

判別分析を用いて，手書き数字を識別する（0から9の10クラスに分類・判別する）実験を行いましょう．


<!---
<b><font color="#ff0000">
注意:
今回の notebook の中には，コードセルを実行すると問題の解答が表示されるようになっている箇所があります．
</font>
</b>
--->


In [ ]:
# いつものいろいろインポート
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set_theme()

# scikit-learn のもろもろ
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix

---
### データの準備



次のセルを実行すると，データを読み込みます．

In [ ]:
# 手書き数字画像データの入手
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/minimnist.npz
rv = np.load('minimnist.npz')
X_train = rv['datL'].astype(float)
y_train = rv['labL']
X_test = rv['datT'].astype(float)
y_test = rv['labT']

K = 10 # クラス数
N_train, D = X_train.shape
N_test, _  = X_test.shape
print(f'K = {K}, D = {D}, N_train = {N_train}, N_test = {N_test}')

このデータセットは，次のようなものです．

- 0 から 9 までの数が書かれた手書き数字の画像を集めたもの．したがって，クラス数 `K` は 10．
- ひとつの画像は，グレイスケールで $28\times 28 = 784$ 画素から成る．したがって，データの次元数 `D` は 784．
- データは合計 6000 枚の手書き数字画像から成る．そのうち `N_train` (=5000) 枚は，モデルのパラメータを推定するために用いるデータ（これを「学習データ」または「訓練データ」という）である．残り `N_test` (=1000) 枚は，モデルの性能を評価するために用いるデータ（これを「テストデータ」という）である（注）．

このデータセットは，機械学習の分野で例題としてよく用いられる MNIST（[Wikipediaの記事](https://ja.wikipedia.org/wiki/MNIST%E3%83%87%E3%83%BC%E3%82%BF%E3%83%99%E3%83%BC%E3%82%B9)） という手書き数字画像のデータセットから一部を抽出したものになっています．

<br>
<hr width="50%" align="left">
<span style="font-size: 75%">
※注: 「学習データ」/「訓練データ」は，「機械学習」分野の用語です．「学習データ」と「テストデータ」を分けて扱う理由は，「機械学習I/II」の授業で学べます．
</span>

次のコードセルを実行すると，データの一部を画像として表示させることができます．

In [ ]:
# 学習データの一部を可視化
nrow, ncol = 4, 10
fig, ax = plt.subplots(nrow, ncol, figsize=(8, 4))
for i in range(nrow):
    for j in range(ncol):
        n = i * ncol + j
        ax[i, j].imshow(X_train[n].reshape(28, 28), vmin=0, vmax=255, cmap='gray')
        ax[i, j].axis('off')
        ax[i, j].set_title(f'{y_train[n]}')
fig.tight_layout()
plt.show()

画像の上に付された数字は，その画像のクラス番号（＝その画像に写った数字がいくつであるか）を表します．

---
### 実験1: 線形判別分析

学習データに線形判別分析を適用してみよう．学習データ/テストデータそれぞれどの程度正しく識別できるでしょうか．

ここでは，scikit-learn の線形判別分析クラス [sklearn.discriminant_analysis.LinearDiscriminantAnalysis](https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html) を用います．

In [ ]:
# 学習データを用いて線形判別分析のパラメータを推定
lda = LinearDiscriminantAnalysis(store_covariance=True)
lda.fit(X_train, y_train)

# 推定されたパラメータの shape を表示
mu = lda.means_
cov = lda.covariance_
print(f'mu.shape: {mu.shape}')
print(f'cov.shape: {cov.shape}')

# 平均を可視化
fig, ax = plt.subplots(1, 10, figsize=(8, 1))
for k in range(10):
    ax[k].imshow(mu[k].reshape(28, 28), vmin=0, vmax=255, cmap='gray')
    ax[k].axis('off')
    ax[k].set_title(f'{k}')
fig.tight_layout()
plt.show()

線形判別分析の場合，クラス毎にそのクラスに属するデータに正規分布を当てはめます．ただし，分散共分散行列は，クラスによらず同一とします．したがって，推定されるパラメータは次の通りです．

- 平均: $K$ 個の $D$ 次元ベクトル
- 分散共分散行列: ひとつの$D\times D$ 行列

上のコードセルの場合，推定された平均および分散共分散行列は，それぞれ`mu` および `cov` という変数名の NumPy array に格納されています．いまは $K=10, D = 784$ ですので，コードセルを実行して表示されたこれらの shape が上記の説明の通りになっていることが確認できます．

画像は，推定された平均を可視化したものです．同じ数字でも多様なバリエーションがあるため，平均をとると少々ぼけた画像になっています．

それでは，学習データとテストデータのそれぞれについて，どのくらい正しく識別できるか調べましょう．与えたデータのうち予測が正解だったものの割合を求めます．混同行列も求めています（混同行列については↓のコメント参照）．

In [ ]:
# 正答率
print(f'学習データの正答率   = {lda.score(X_train, y_train)}')
print(f'テストデータの正答率 = {lda.score(X_test, y_test)}')
print()
# 混同行列  confusion[i, j] は，正解がクラス i で予測がクラス j だったものの数
y_pred = lda.predict(X_test)
confusion = confusion_matrix(y_test, y_pred)
print(confusion)

学習データでは 9 割以上正しく識別できていますが，テストデータでは少し間違えることが多くなる結果となりました．特に 2 を 8 に間違えるケースと，9 を 4 に間違えるケースが多いようです（ともに11件ずつ）．

In [ ]:
# 誤答例
idx = y_test != y_pred
XX = X_test[idx, :]
n = XX.shape[0]
nrow, ncol = 4, 10
print(f'誤答 {n} 件のうち {nrow *ncol} 件を表示')
fig, ax = plt.subplots(nrow, ncol, figsize=(8, 4))
for i in range(nrow):
    for j in range(ncol):
        n = i * ncol + j
        ax[i, j].imshow(XX[n].reshape(28, 28), vmin=0, vmax=255, cmap='gray')
        ax[i, j].axis('off')
        ax[i, j].set_title(f'{y_test[idx][n]} / {y_pred[idx][n]}')
fig.tight_layout()
plt.show()

画像の上の数字 $x/y$ は，正解が $x$ で予測が $y$ だったことを示します．

---
### 実験2: 二次判別分析

同じことを二次判別分析でもやってみよう．結果に違いはあるでしょうか．

[sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis](https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis.html) を用います．

In [ ]:
# 学習データを用いて二次判別分析のパラメータを推定
qda = QuadraticDiscriminantAnalysis(store_covariance=True)
qda.fit(X_train, y_train)

# 推定されたパラメータの shape を表示
mu = qda.means_
cov = np.array(qda.covariance_)
print(f'mu.shape: {mu.shape}')
print(f'cov.shape: {cov.shape}')

上のコードセルを実行すると，`Variables are collinear` という警告（Warning）メッセージが出るはずです．この警告は，変数に多重共線性があるよ，と言っています（「多重共線性」は，重回帰分析で出てきましたね）．二次判別分析ではクラス毎に分散共分散行列を求めるので推定せねばならないパラメータの数が非常に多いため，推定に問題が生じたようです．このことは，予測結果に悪い影響を及ぼしている可能性があります．

実際，二次判別分析で推定されるパラメータは次の通りです．

- 平均: $K$ 個の $D$ 次元ベクトル
- 分散共分散行列: $K$ 個の $D\times D$ 行列

$K = 10, D = 784$ として，線形判別・二次判別それぞれの場合に推定すべきパラメータの総数を求めると

- 線形判別分析: $KD + \frac{1}{2}D(D+1) = 7840 + 307720 = 315560$ 個
- 二次判別分析: $KD + K \times \frac{1}{2}D(D+1) = 7840 + 3077200 = 3085040$ 個

となり，二次判別では線形判別の約10倍，300万個の値を推定せねばならないことが分かります．

正答率を見てみると...

In [ ]:
# 正答率
print(f'学習データの正答率   = {qda.score(X_train, y_train)}')
print(f'テストデータの正答率 = {qda.score(X_test, y_test)}')
print()
# 混同行列  confusion[i, j] は，正解がクラス i で予測がクラス j だったものの数
y_pred = qda.predict(X_test)
confusion = confusion_matrix(y_test, y_pred)
print(confusion)

学習データについてはほぼ正解しているものの，テストデータの正答率は線形判別よりも大幅に低くなってしまいました．やはり，パラメータ推定の問題が悪影響を及ぼしたようです．

ここで扱っているデータは，$D = 784$ と次元数が大きいわりに，データ数が多くありません．そのため，線形判別よりもパラメータ数のずっと多い二次判別の方がうまくいかない結果となったようです．

notebookB では，単純な例で，線形判別よりも二次判別の方がよい結果を得られる場合を見ましたが，ここまでの話から分かるように，パラメータ数の多い複雑な手法の方が常によい結果を得られるわけではありません．この問題のもう少しちゃんとした説明と，その対処法については，「機械学習I/II」で登場します．

---
### 実験3: 主成分分析＋二次判別分析

データの次元数が大きいせいでパラメータ数が多いために二次判別がうまくいかなかったのなら，次元数を削減したデータでやれば改善できるかもしれません．主成分分析による次元削減を組み合わせたらどうでしょうか．

[sklearn.decomposition.PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) を用いることにします．$D = 784$ ですが，変換後の次元数は，累積寄与率が 0.9 以上となる最小の数となるようにします．

In [ ]:
# PCA  データの平均と，分散共分散行列の固有ベクトルを
#      固有値の大きい方から順にならべた行列を求める
pca = PCA(n_components=0.9)
pca.fit(X_train)
H = pca.n_components_ # 変換後の次元数
print(f'H = {H}')

# 得られた平均と固有ベクトルを用いて次元削減の変換を実行
# y = U^T(x - mu)
XX_train = pca.transform(X_train)

# 次元削減した学習データを用いて二次判別分析のパラメータを推定
qda = QuadraticDiscriminantAnalysis(store_covariance=True)
qda.fit(XX_train, y_train)

# 推定されたパラメータの shape を表示
mu = qda.means_
cov = np.array(qda.covariance_)
print(f'mu.shape: {mu.shape}')
print(f'cov.shape: {cov.shape}')

主成分分析を適用して次元削減を行った結果，データの次元数は 86 まで削減されました．

In [ ]:
# テストデータも変換
XX_test = pca.transform(X_test)
# 正答率
print(f'学習データの正答率   = {qda.score(XX_train, y_train)}')
print(f'テストデータの正答率 = {qda.score(XX_test, y_test)}')
print()
# 混同行列  confusion[i, j] は，正解がクラス i で予測がクラス j だったものの数
y_pred = qda.predict(XX_test)
confusion = confusion_matrix(y_test, y_pred)
print(confusion)

主成分分析を組み合わせることで，元データを使った線形判別の場合よりも高いテスト正答率が得られました．

In [ ]:
# 誤答例
idx = y_test != y_pred
XX = X_test[idx, :]
n = XX.shape[0]
nrow, ncol = 4, 10
print(f'誤答 {n} 件のうち {nrow *ncol} 件を表示')
fig, ax = plt.subplots(nrow, ncol, figsize=(8, 4))
for i in range(nrow):
    for j in range(ncol):
        n = i * ncol + j
        ax[i, j].imshow(XX[n].reshape(28, 28), vmin=0, vmax=255, cmap='gray')
        ax[i, j].axis('off')
        ax[i, j].set_title(f'{y_test[idx][n]} / {y_pred[idx][n]}')
fig.tight_layout()
plt.show()

ここでは，線形判別分析と二次判別分析によって，手書き数字画像の識別を行いました．行った実験の中では，主成分分析を組み合わせた二次判別が最も高い正答率を示しました．この授業では，判別/分類/識別の問題にこれ以上は踏み込みませんが，「機械学習I/II」では，様々な機械学習手法を紹介します．判別分析よりずっと高い正答率を得られるものも出てきます．